<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/HyperParameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Machine-Learning.git cloned-repo
%cd cloned-repo
!ls

In [0]:
#import data
import pandas as pd

bc_data = pd.read_csv('breast_cancer_data.csv')
#view top 10 rows
bc_data.head(10)


In [0]:
bc_data.columns
bc_data.drop(index=1,columns='Unnamed: 32')

In [0]:
#convert diagnosis to 0= benign, 1= maglignant
#bc_data.iloc[:,1]
def convert_diagnosis(x):
  if(x=='M'):
    return '1'
  if(x=='B'):
    return '0'

bc_data['c_diagnosis'] = bc_data['diagnosis'].apply(convert_diagnosis)

In [0]:
bc_data.columns

In [0]:
bc_data.pop('diagnosis')
bc_data.pop('Unnamed: 32')
bc_data.tail(20)

In [0]:
#Split data into attributes and class
X = bc_data.drop(['c_diagnosis'],axis=1)
y = bc_data['c_diagnosis']

#perform training and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#Dummy Classifier
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy= 'most_frequent').fit(X_train,y_train)
y_pred = clf.predict(X_test)

#Distribution of y test
print('y actual : \n' +  str(y_test.value_counts()))

#Distribution of y predicted
print('y predicted : \n' + str(pd.Series(y_pred).value_counts()))

In [0]:
# Model Evaluation metrics 
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
#print('Recall Score : ' + str(recall_score(y_test,y_pred)))
#print('F1 Score : ' + str(f1_score(y_test,y_pred)))

#Dummy Classifier Confusion matrix
from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

In [0]:
#Logistic regression
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_train,y_train)
y_pred = clf.predict(X_test)

# Model Evaluation metrics 
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

#Logistic Regression Classifier Confusion matrix
from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))
